# Calculate entity end dates: brownfield-land
**Author**:  Greg Slater <br>
**Date**:  13 Jan 2025 <br>
**Scope**: `dataset` <br>
**Report Type**: Ad-hoc analysis <br>
**Purpose**: Script to identify BFL entities which can be given an end-date on the basis that their references do not appear on the latest resource for a provision.

In [4]:
import pandas as pd
import os
import urllib
import sqlite3
from datetime import datetime
import numpy as np

td = datetime.today().strftime('%Y-%m-%d')

pd.set_option("display.max_rows", 100)

data_dir = "../data/"
os.makedirs(data_dir, exist_ok=True)
out_dir = "../output/"
os.makedirs(out_dir, exist_ok=True)

## Problem organisations to exclude
# This is a list of organisations who we know supply multiple endpoints for multiple areas - we need to exclude these as this process only works for 
# provisions where there is one dataset updated over subsequent endpoints and resources, rather than multiple datasets.

orgs_to_exclude = ["local-authority:BUC"]


In [5]:
def datasette_query(db, sql_string):
    params = urllib.parse.urlencode({
        "sql": sql_string,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{db}.csv?{params}"
    df = pd.read_csv(url)
    return df

def get_all_organisations():
    q = """
        select organisation, name, entity as organisation_entity
        from organisation
        where end_date = ""
        """
    return datasette_query("digital-land", q)

FILES_URL = 'https://datasette.planning.data.gov.uk/'

def download_dataset(dataset, output_dir_path, overwrite=False):
    dataset_file_name = f'{dataset}.db'
    
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)
    
    output_file_path = os.path.join(output_dir_path, dataset_file_name)

    if overwrite is False and os.path.exists(output_file_path):
        return
    
    final_url = os.path.join(FILES_URL, dataset_file_name)
    print(f'downloading data from {final_url}')
    print(f'to: {output_file_path}')
    urllib.request.urlretrieve(final_url, os.path.join(output_dir_path, dataset_file_name))
    print('download complete')

def query_sqlite(db_path, query_string):

    with sqlite3.connect(db_path) as con:
            
        cursor = con.execute(query_string)
        cols = [column[0] for column in cursor.description]
        results_df = pd.DataFrame.from_records(data=cursor.fetchall(), columns=cols)

    return results_df

In [23]:
# function to get historic endpoints and resources for a provision, along with incrementing resource key
def get_historic_resources(dataset, organisation):

    q = f"""

        WITH endpoint_count as (
            SELECT 
                distinct organisation,
                latest_status, 
                endpoint, 
                endpoint_entry_date, 
                endpoint_end_date, 
                resource, 
                resource_start_date, 
                resource_end_date,
                dense_rank() over (order by endpoint_entry_date, endpoint) as endpoint_no

            FROM reporting_historic_endpoints
            WHERE pipeline = '{dataset}'
            AND organisation = '{organisation}'
            AND (latest_status = 200 OR latest_status = "")
            ORDER BY endpoint_entry_date, resource_start_date
        ),

        endpoint_resource_count as (
            SELECT 
                *,
                CAST(CAST(endpoint_no as string) || "." || dense_rank() over (partition by endpoint_no order by resource_start_date, latest_status) as float) as endpoint_resource_no
            FROM endpoint_count
        )

        SELECT 
            *,
            CASE WHEN resource_end_date = "" then 
                LEAD(endpoint_entry_date) over (order by endpoint_resource_no) 
                ELSE resource_end_date end
                as entity_end_date

        FROM endpoint_resource_count

"""
    r = datasette_query("performance", q)

    return r 

# get_historic_resources("brownfield-land", "local-authority:BST")

In [7]:
def get_resource_references(resources, db_path):

    # take list of resources and db path to get all reference values which have appeared on each resource

    q = """ 

        SELECT distinct fr.resource, f.value 
        FROM fact_resource fr
        INNER JOIN fact f on fr.fact = f.fact
        WHERE 1=1
            AND f.field = "reference"
            AND fr.resource in ({})
    """.format(', '.join(f"'{r}'" for r in resources))

    df = query_sqlite(db_path, q)
    
    return df

## Data Import

In [ ]:
# download performance db
download_dataset("brownfield-land", data_dir, overwrite=True)
bfl_path = os.path.join(data_dir, "brownfield-land.db")

In [9]:
org_lookup = get_all_organisations()

org_dict = dict(zip(org_lookup["organisation"], org_lookup["organisation_entity"]))

In [ ]:
prov_bfl_all = datasette_query(
    "digital-land",
    """
    SELECT organisation
    FROM provision
    WHERE dataset = "brownfield-land"
    AND end_date = ""
    """
)

# remove organisations to exclude
prov_bfl = prov_bfl_all[~prov_bfl_all["organisation"].isin(orgs_to_exclude)].copy()

print(len(prov_bfl_all))
print(len(prov_bfl))


## Run

In [ ]:

all_out = []

for org in prov_bfl["organisation"]:
    
    # get all valid resources
    res_hist = get_historic_resources("brownfield-land", org)

    print("---------------------------------------")
    print(org)
    print(f"Number of endpoint & resource records found: {len(res_hist)}")

    if len(res_hist) > 1:

        # get entities
        org_entities = query_sqlite(bfl_path, 
            f"""
                SELECT * 
                FROM entity
                WHERE organisation_entity = {org_dict[org]}
                and end_date = ""
            """)

        print(f"Number of live entities: {len(org_entities)}")

        # get all reference values for resources
        res_refs = get_resource_references(
            res_hist["resource"].drop_duplicates(), 
            bfl_path)
        
        # join resources to list of reference values per resource
        org_res_refs = res_hist.merge(
            res_refs,
            how = "left",
            on = "resource"
        )

        # get the endpoint-resource count code for the most recent resource each reference has appeared in 
        ref_latest_res = org_res_refs.groupby(["value"], as_index = False).agg(
            endpoint_resource_no = ("endpoint_resource_no", "max")
        )

        print(len(ref_latest_res))
        
        # join back to the full resource table using the endpoint_resource_no to get the end-date of the most recent
        ref_latest_res_dated = ref_latest_res.merge(
            org_res_refs[["resource", "endpoint_resource_no", "value", "entity_end_date"]],
            how = "left",
            on = ["value", "endpoint_resource_no"]
        )

        # join from entity table to our reference and resource calcs
        entity_ref_latest_res_dated = org_entities[["entity", "organisation_entity", "reference", "end_date"]].merge(
            ref_latest_res_dated,
            how = "left",
            left_on = "reference",
            right_on = "value"
        )

        entity_ref_latest_res_dated.drop("value", axis=1, inplace=True)

        # add flags for old and mystery entities
        max_ep_res = org_res_refs["endpoint_resource_no"].max()
        entity_ref_latest_res_dated["old_entity"] = np.where(entity_ref_latest_res_dated["endpoint_resource_no"] != max_ep_res, True, False)
        entity_ref_latest_res_dated["mystery_entity"] = np.where(entity_ref_latest_res_dated["endpoint_resource_no"].isnull(), True, False)

        all_out.append(entity_ref_latest_res_dated)




In [ ]:
all_out_df = pd.concat(all_out)
print(len(all_out_df))

all_out_df["organisation_entity"] = all_out_df["organisation_entity"].astype(int)
all_out_df = all_out_df.merge(
    org_lookup[["organisation_entity", "organisation"]],
    on = "organisation_entity"
)

unique_ent_test = len(all_out_df) == len(all_out_df["entity"].drop_duplicates())
print(f"Is the output table unique by entity: {unique_ent_test}")

# all_out_df["mystery_entity"] = np.where(all_out_df["endpoint_resource_no"].isnull(), True, False)
# all_out_df["old_entity"] = np.where(all_out_df["entity_end_date"].notnull(), True, False)

all_out_df.sort_values(["organisation", "endpoint_resource_no"], inplace=True, ascending= [True, False])

all_out_df.to_csv(os.path.join(data_dir, f"old_entity_results_all_{td}.csv"), index = False)

print(len(all_out_df))
all_out_df.head()

In [ ]:
print(len(all_out_df[all_out_df["old_entity"]]))
print(len(all_out_df[all_out_df["mystery_entity"]]))

In [36]:
# old entities are those flagged as old, and not flagged as mystery
old_entities_out = all_out_df[
    (all_out_df["old_entity"]) & (~all_out_df["mystery_entity"])][["entity", "entity_end_date"]]

old_entities_out.rename(columns={"entity_end_date":"end-date"}, inplace=True)

old_entities_out.to_csv(os.path.join(out_dir, "brownfield-land-end-date.csv"))